In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [ ]:
# Line parameters
R = 3e-3
L = 1e-3
G = 0
C = 1e3

c_squared = 1. / (L*C)
alpha = (G / C)
beta = (R / L)

In [ ]:
# Domain stuff
X = 1.0
T = 1.0

K = 99
delta_x = 